In [1]:
import uproot
from glob import glob
import json 
import os
from tqdm import tqdm
import numpy as np
import pandas as pd

In [2]:
pi0_dir = '/clusterfs/ml4hep/mpettee/ml4pions/data/user.angerami.mc16_13TeV.900246.PG_singlepi0_logE0p2to2000.e8312_e7400_s3170_r12383.v01-45-gaa27bcb_OutputStream/'
pion_dir = '/clusterfs/ml4hep/mpettee/ml4pions/data/user.angerami.mc16_13TeV.900247.PG_singlepion_logE0p2to2000.e8312_e7400_s3170_r12383.v01-45-gaa27bcb_OutputStream/'
pi0_files = sorted(glob(pi0_dir+"*.root"))
pion_files = sorted(glob(pion_dir+"*.root"))

Check out one file as a test...

In [3]:
a = uproot.open(pion_files[0])["EventTree"].arrays(library = "np")
a.keys()

dict_keys(['runNumber', 'eventNumber', 'lumiBlock', 'coreFlags', 'mcEventNumber', 'mcChannelNumber', 'mcEventWeight', 'nTruthPart', 'G4PreCalo_n_EM', 'G4PreCalo_E_EM', 'G4PreCalo_n_Had', 'G4PreCalo_E_Had', 'truthVertexX', 'truthVertexY', 'truthVertexZ', 'truthPartPdgId', 'truthPartStatus', 'truthPartBarcode', 'truthPartPt', 'truthPartE', 'truthPartMass', 'truthPartEta', 'truthPartPhi', 'nTrack', 'trackPt', 'trackP', 'trackMass', 'trackEta', 'trackPhi', 'trackNumberOfPixelHits', 'trackNumberOfSCTHits', 'trackNumberOfPixelDeadSensors', 'trackNumberOfSCTDeadSensors', 'trackNumberOfPixelSharedHits', 'trackNumberOfSCTSharedHits', 'trackNumberOfPixelHoles', 'trackNumberOfSCTHoles', 'trackNumberOfInnermostPixelLayerHits', 'trackNumberOfNextToInnermostPixelLayerHits', 'trackExpectInnermostPixelLayerHit', 'trackExpectNextToInnermostPixelLayerHit', 'trackNumberOfTRTHits', 'trackNumberOfTRTOutliers', 'trackChiSquared', 'trackNumberDOF', 'trackD0', 'trackZ0', 'trackEta_PreSamplerB', 'trackPhi_PreS

In [ ]:
# [var for var in a.keys() if "Eta" in var]

Now apply all the cuts...

In [27]:
import math 
def delta_phi(phi1, phi2):
    return (phi1 - phi2 + math.pi) % (2*math.pi) - math.pi

def apply_cuts(arrays):
    df = pd.DataFrame(arrays)

    ### Single-track, multi-cluster 
    df = df[(df.nTrack == 1)]
    dR_pass = []
    deltaR_list = []
    for row in df.index:
        try:
            deltaR = np.sqrt((df['cluster_Eta'][row].astype('float') - df['trackEta'][row].astype('float'))**2 + 
                               delta_phi(df['cluster_Phi'][row].astype('float'), df['trackPhi'][row].astype('float'))**2)
        except:
            deltaR = np.array(999)
        deltaR_list.append(deltaR)
        dR_pass.append(deltaR < 1.2)
    df["deltaR"] = deltaR_list
    df["dR_pass"] = dR_pass
    df["event_number"] = df.index
    df.reset_index(inplace=True)
    indices_pass = [] 
    for row in df.index: # kill all events with no clusters passing the delta R cut; deal with individual clusters at the training stage
        if len(df['trackPt'][row]) > 1:
            continue
        elif df.dR_pass[row].sum() > 0:
            indices_pass.append(row)
    df = df.iloc[indices_pass]
    df = df[(df.trackPt < 10**5)] # Track pT cut 
    return df

In [20]:
# use a pared-down list of variables 
variables = ['cluster_cell_E', 'cluster_cell_ID',
             'trackPt','trackD0','trackZ0',
             'trackEta_EMB2','trackPhi_EMB2',
             'trackEta_EME2','trackPhi_EME2',
             'trackEta','trackPhi',
             'nCluster','nTrack','truthPartE', 'truthPartPt',
             'cluster_ENG_CALIB_TOT','cluster_E','cluster_Eta','cluster_Phi',
             'cluster_EM_PROBABILITY','cluster_E_LCCalib','cluster_HAD_WEIGHT']

In [21]:
# for file in tqdm(pi0_files):
#     prefix = file.split("/")[:-2]
#     number = file.split("000")[-1][:-5]
#     folder = os.path.join("/".join(prefix), "onetrack_multicluster", "pi0_files")
#     os.makedirs(folder, exist_ok=True)
#     npy_filename = os.path.join(folder, str(number)+".npy")
#     a = uproot.open(file)["EventTree"].arrays(variables, library = "np")
#     df = apply_cuts(a)
#     a_cuts = df.to_dict('list')
#     np.save(npy_filename, a_cuts) 

In [29]:
for file in tqdm(pion_files):
    prefix = file.split("/")[:-2]
    number = file.split("000")[-1][:-5]
    folder = os.path.join("/".join(prefix), "onetrack_multicluster", "pion_files")
    os.makedirs(folder, exist_ok=True)
    npy_filename = os.path.join(folder, str(number)+".npy")
    a = uproot.open(file)["EventTree"].arrays(variables, library = "np")
    df = apply_cuts(a)
    a_cuts = df.to_dict('list')
    np.save(npy_filename, a_cuts) 

  0%|          | 0/500 [00:01<?, ?it/s]


KeyboardInterrupt: 

### Testing

In [ ]:
df = apply_cuts(a)

In [ ]:
all_drs = []
dfs = []
for file in tqdm(pion_files[:10]):
    prefix = file.split("/")[:-2]
    number = file.split("000")[-1][:-5]
    folder = os.path.join("/".join(prefix), "onetrack_multicluster", "pion_files")
    os.makedirs(folder, exist_ok=True)
    npy_filename = os.path.join(folder, str(number)+".npy")
    a = uproot.open(file)["EventTree"].arrays(variables, library = "np")
    df = apply_cuts(a)
    list = df.deltaR.to_numpy().flatten()
    drs = np.concatenate(list, axis=0)
    all_drs.append(drs)
    dfs.append(df)
    
import matplotlib.pyplot as plt
all_drs = np.concatenate(all_drs, axis=0)

plt.figure(dpi=200)
plt.hist(all_drs, bins=np.linspace(0.5,1.5,30));
plt.xlabel(r"$\Delta R$(cluster,track)")

plt.figure(dpi=200)
plt.hist(all_drs, bins=np.linspace(0.5,1.5,30));
plt.yscale("log")
plt.xlabel(r"$\Delta R$(cluster,track)")

In [ ]:
df = pd.concat([df for df in dfs])

In [ ]:
plt.figure(dpi=150)
plt.hist(np.concatenate(df.trackPt.to_numpy().flatten(), axis=0), bins=10);
plt.xlabel(r"Track $p_T$")
plt.yscale("log")